In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests
import datetime

# Function to fetch data from CoinGecko
def fetch_data(crypto_id):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': '90',
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['crypto'] = crypto_id.capitalize()
    
    return df

# Get data for selected cryptocurrencies
cryptos = ['bitcoin', 'ethereum', 'litecoin']
df_list = [fetch_data(crypto) for crypto in cryptos]
df = pd.concat(df_list)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Crypto Market Dashboard"),
    
    dcc.Dropdown(
        id='crypto-filter',
        options=[{'label': crypto.capitalize(), 'value': crypto} for crypto in cryptos],
        value=[cryptos[0]],
        multi=True
    ),
    
    dcc.DatePickerRange(
        id='date-picker',
        start_date=df['timestamp'].min().date(),
        end_date=df['timestamp'].max().date(),
        display_format='YYYY-MM-DD',
        start_date_placeholder_text="Start Date",
        end_date_placeholder_text="End Date"
    ),
    
    dcc.Graph(id='price-chart'),
    
    html.Div(id='market-stats')
])

# Callback to update the graph
@app.callback(
    Output('price-chart', 'figure'),
    [Input('crypto-filter', 'value'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date')]
)
def update_graph(selected_cryptos, start_date, end_date):
    if not selected_cryptos:
        return {}
    
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    filtered_df = df[(df['crypto'].isin([crypto.capitalize() for crypto in selected_cryptos])) & 
                     (df['timestamp'] >= start_date) & 
                     (df['timestamp'] <= end_date)]
    
    fig = px.line(filtered_df, x='timestamp', y='price', color='crypto', title="Crypto Prices Over Time")
    
    # Customizing colors and layout
    fig.update_traces(line=dict(width=2))
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        title_font=dict(size=24),
        font=dict(color='black')
    )
    
    return fig

# Callback to update market stats
@app.callback(
    Output('market-stats', 'children'),
    [Input('crypto-filter', 'value')]
)
def update_stats(selected_cryptos):
    if not selected_cryptos:
        return "Select a cryptocurrency to view market stats."
    
    stats = []
    for crypto in selected_cryptos:
        latest_data = df[df['crypto'] == crypto.capitalize()].iloc[-1]
        stats.append(html.P(f"{crypto.capitalize()} - Price: ${latest_data['price']:.2f}"))
    
    return stats

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
